#**Машинное обучение ИБ-2025**
Верещагина Ангелина Леонидовна, БИБ 231

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.




In [2]:
!pip install folium

Загрузим данные из csv файла в датафрейм.

In [3]:
df = pd.read_csv('input_data.csv', sep=';')

Отобразим на карте координаты наших построек.

In [4]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# 1. Подготовим данные для обработки моделями машинного обучения.

География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [5]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Рассчитывает расстояние по большой окружности (в километрах) между двумя точками
    на поверхности Земли, заданными широтой/долготой в градусах,
    используя формулу Haversine
    """
    R = 6371  # Радиус Земли в километрах (среднее значение) - используется для перевода углового расстояния в км

    # math.radians переводит градусы -> радианы, все тригонометрические функции в Python работают в радианах
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Разницы координат в радианах
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

 # Формула Haversine

    # a = sin^2(dlat/2) + cos(lat1)*cos(lat2)*sin^2(dlon/2)
    # c = 2 * atan2(sqrt(a), sqrt(1-a))
    # distance = R * c
    # a — промежуточная величина, связанная с "хаверсином" угла между точками,
    # c — центральный угол (в радианах), distance — дуговое расстояние на сфере
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c # итоговое расстояние в километрах
    return distance

# Примеры центров (широта, долгота) — в градусах
moscow_center = (55.7558, 37.6173)
saint_petersburg_center = (59.9343, 30.3351)

# Порог в километрах — точка считается "внутри города", если ближе или равно этому расстоянию
distance_threshold = 20

# Создание новых булевых признаков в DataFrame

# Что делает этот apply?
# для каждой строки DataFrame вызывает функцию, считает расстояние от точки в строке
# до центра города и ставит True/False в новый столбец в зависимости от порога
df['is_Moscow'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], moscow_center[0], moscow_center[1]) <= distance_threshold, axis=1)
df['is_Saint_Peterburg'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], saint_petersburg_center[0], saint_petersburg_center[1]) <= distance_threshold, axis=1)

In [6]:
print("Value counts for is_Moscow:")
print(df['is_Moscow'].value_counts())

print("\nValue counts for is_Saint_Peterburg:")
print(df['is_Saint_Peterburg'].value_counts())

Value counts for is_Moscow:
is_Moscow
False    882957
True      83038
Name: count, dtype: int64

Value counts for is_Saint_Peterburg:
is_Saint_Peterburg
False    899585
True      66410
Name: count, dtype: int64


В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [7]:
columns_to_drop = ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id']
df = df.drop(columns=columns_to_drop)
display(df.head())

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,False,False


 Добавим признаки, связанные с датой и отношением уровня к количеству уровней (level/levels)


In [8]:
# преобразую столбец 'date' из строкового формата
# в тип datetime, чтобы можно было выполнять операции с датами

df['date'] = pd.to_datetime(df['date'])
first_observation_date = df['date'].min() # нашли самую раннюю дату в наборе данных

# создадим новый числовой признак, количество дней, прошедших с первой записи
# так алгоритмы смогут понять временную динамику (чем новее объявление, тем больше days_since_first_observation)

df['days_since_first_observation'] = (df['date'] - first_observation_date).dt.days

# еще один новый признак — отношение этажа к общему числу этажей
# если квартира на 5 этаже из 20, то ratio = 0.25
df['level_to_levels_ratio'] = df['level'] / df['levels']

# может быть что 'levels' = 0, что приведет к делению на ноль → inf или NaN

df['level_to_levels_ratio'] = df['level_to_levels_ratio'].replace([np.inf, -np.inf], np.nan)
# заменим такие значения на NaN, чтобы избежать бесконечностей в данных
df['level_to_levels_ratio'] = df['level_to_levels_ratio'].fillna(0)

# удаляю исходный столбец 'date'потому что теперь он не нужен
df = df.drop(columns=['date'])

# нормализация числовых признаков
from sklearn.preprocessing import StandardScaler

# выбираем список числовых столбцов, которые нужно стандартизировать
# масштабирование помогает привести признаки к единой шкале (среднее 0, стандартное отклонение 1)
# что важно для алгоритмов, чувствительных к масштабу признаков (в нашем случ линейная регрессия)
numerical_cols_to_scale = ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'days_since_first_observation', 'level_to_levels_ratio']

scaler = StandardScaler()
df[numerical_cols_to_scale] = scaler.fit_transform(df[numerical_cols_to_scale])
# fit_transform: сначала вычисляет среднее и стандартное отклонение, потом масштабирует


display(df.head())

,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg,days_since_first_observation,level_to_levels_ratio
0,-0.006501,1.560669,2.583524,-0.630622,-0.858112,0.122394,0,False,False,-2.193078,-0.298681
1,-0.008046,-0.278873,-0.931082,-0.630622,-0.758456,0.297715,0,False,False,-2.193078,1.355285
2,0.006224,-0.462827,0.150335,1.092634,1.160834,0.473037,3,False,False,-2.193078,-0.863256
3,-0.005500,-0.646781,-0.931082,1.092634,1.050106,0.385376,0,False,False,-2.193078,0.073462
4,-0.006426,-0.830736,-1.201436,-0.630622,-0.869185,0.385376,3,False,False,-2.193078,0.287099


Модели машинного обучения не умеют работать с датами напрямую.
Но если представить дату в виде численного признака (например, сколько дней прошло с начала наблюдений), алгоритм сможет улавливать временные закономерности, например, что цена жилья растёт со временем.

Многие модели также чувствительны к масштабу признаков.
Например, признак price может быть в тысячах, а level_to_levels_ratio — от 0 до 1.
Если не масштабировать, алгоритм будет считать, что price важнее просто потому, что его значения больше.

Какие колонки категориальные? Какие числовые?

In [9]:
# Определяю категориальные и числовые признаки

categorical_cols = ['building_type', 'is_Moscow', 'is_Saint_Peterburg']
numerical_cols = ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area'] # 'date' will be handled later

print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder переводит категориальные переменные в числовые векторные признаки
# - handle_unknown='ignore' — если в тестовых данных встретится новая категория, не введенная при обучении, то она будет проигнорирована
# - sparse_output=False — возвращает обычный NumPy-массив, а не разреженную матрицу
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_categorical_data = encoder.fit_transform(df[categorical_cols])

# fit_transform обучает энкодер (запоминает все категории) и сразу применяет кодирование
# в рез-те матрица, где каждая категория превращается в отдельную колонку с 0/1
encoded_categorical_data = encoder.fit_transform(df[categorical_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_cols))

# Объединяем полученный DataFrame с исходным без старых категориальных столбцов
# drop(columns=categorical_cols) — удаляем исходные категориальные признаки, которые больше не нужны
# concat — объединяем по горизонтали
df = pd.concat([df.drop(columns=categorical_cols), encoded_categorical_df], axis=1)

display(df.head())

Categorical columns: ['building_type', 'is_Moscow', 'is_Saint_Peterburg']
Numerical columns: ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area']


,price,level,levels,rooms,area,kitchen_area,days_since_first_observation,level_to_levels_ratio,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_False,is_Moscow_True,is_Saint_Peterburg_False,is_Saint_Peterburg_True
0,-0.006501,1.560669,2.583524,-0.630622,-0.858112,0.122394,-2.193078,-0.298681,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.008046,-0.278873,-0.931082,-0.630622,-0.758456,0.297715,-2.193078,1.355285,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.006224,-0.462827,0.150335,1.092634,1.160834,0.473037,-2.193078,-0.863256,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,-0.005500,-0.646781,-0.931082,1.092634,1.050106,0.385376,-2.193078,0.073462,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,-0.006426,-0.830736,-1.201436,-0.630622,-0.869185,0.385376,-2.193078,0.287099,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


# 2. Реализуем методы регрессии

Реализуем класс **KNNRegressor**, который должен делать регрессию методом k ближайших соседей.

In [10]:
from sklearn.metrics import pairwise_distances

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        """
        ф-я конструктор класса KNNRegressor.

        n_neighbors — количество ближайших соседей, которые будем учитывать при предсказании
        metric — метрика расстояния, по которой измеряется близость объектов
        """
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        """
        ф-я запоминает обучающую выборку - признаки и целевые значения
            X: Матрица признаков для обучения
            y: Вектор целевых значений
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """
        ф-я предсказывает значения для новых объектов с помощью KNN-регрессии.

        1. для каждой новой точки x_test вычисляем расстояния до всех точек обучающей выборки
        2. находим индексы k ближайших соседей (с минимальными расстояниями)
        3. берём их целевые значения (y_train) и усредняем
        4. среднее значение и будет предсказанием

        """
        predictions = [] # сюда будем сохранять предсказания для каждой точки X
        for x_test in X:
            # Вычисляем расстояния от x_test до всех точек X_train
            distances = pairwise_distances([x_test], self.X_train, metric=self.metric)[0]
            # pairwise_distances возвращает матрицу расстояний размером [1, n_train]
            # берём [0], чтобы получить одномерный массив всех расстояний

            # Сортируем расстояния и выбираем индексы k ближайших соседей
            nearest_neighbors_indices = np.argsort(distances)[:self.n_neighbors]

            # Извлекаем значения целевой переменной для этих соседей
            nearest_neighbors_y = self.y_train[nearest_neighbors_indices]

            # Вычисляем предсказание как среднее значение по соседям
            prediction = np.mean(nearest_neighbors_y)
            predictions.append(prediction)

        return np.array(predictions)



KNN (k-Nearest Neighbors) — метод ближайших соседей.
Он относится к инстанс-базовым (instance-based) алгоритмам машинного обучения,
где модель не обучается явно, а запоминает обучающие примеры.


Чтобы предсказать значение для новой точки
𝑥, нужно:

Найти
𝑘 ближайших точек из обучающего набора по некоторой метрике (например, евклидовой).

Взять их значения целевой переменной
𝑦.

Усреднить их или взять наиболее частое значение.


Реализуем класс **LinearRegression**, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Использую градиент для оптимизации функции потерь MSE.

In [11]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.v = None  # For Momentum
        self.r = None  # For AdaGrad

    def fit(self, X, y):
        """
        Обучает модель линейной регрессии, используя выбранный метод оптимизации
            X — матрица признаков
            y — вектор целевых значений
        """
        # получаем размерность данных
        n_samples, n_features = X.shape
        # инициализация весов и смещения нулями
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Инициализация дополнительных переменных для выбранных методов оптимизации

        if self.optimization == 'Momentum':
            self.v = np.zeros(n_features)
        elif self.optimization == 'AdaGrad':
            self.r = np.zeros(n_features)

        for i in range(self.max_iter):
            # Предсказания модели y_pred = X * w + b
            y_pred = self.predict(X)

            # Вычисляем градиенты функции потерь (MSE)
            # Формула для градиента по весам
            # dw = (1/n) * Xᵀ * (y_pred - y)
            dw = (1/n_samples) * np.dot(X.T, (y_pred - y))
            # Градиент по смещению (bias):
            db = (1/n_samples) * np.sum(y_pred - y)

            # Обновляем веса в зависимости от выбранного метода оптимизации
            if self.optimization == 'SGD':
              # Классический стохастический градиентный спуск
                # Просто двигаемся против градиента с постоянным шагом
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
            elif self.optimization == 'Momentum':
              # --- Метод Momentum ---
                # Добавляем "инерцию" при обновлении весов, чтобы сгладить колебания

                self.v = self.decay_rate * self.v + (1 - self.decay_rate) * dw
                self.weights -= self.learning_rate * self.v
# bias можно тоже корректировать с Momentum, хотя чаще делают обычное обновление

                self.bias -= self.learning_rate * db
            elif self.optimization == 'AdaGrad':
              # Накопление квадратов градиентов для каждого признака
                self.r += dw**2
              # Масштабирование шага: часто встречающиеся признаки получают меньший шаг

                self.weights -= (self.learning_rate / (np.sqrt(self.r + self.epsilon))) * dw
                self.bias -= (self.learning_rate / (np.sqrt(np.sum(db**2) + self.epsilon))) * db
            else:
                raise ValueError("Invalid optimization method. Choose from 'SGD', 'Momentum', 'AdaGrad'.")

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


Этот кусок кода отвечает за разделение данных на обучающую и тестовую выборки,
здесь мы закладываем основу для корректного обучения и оценки модели



In [12]:
from sklearn.model_selection import train_test_split
# Разделяем признаки и целевую переменную
X = df.drop(columns=['price']) # Все столбцы, кроме 'price' — это признаки (факторы)
y = df['price']  # Целевая переменная — цена квартиры

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 20% данных уйдут на тест, 80% — на обучение

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (772796, 18)
Shape of X_test: (193199, 18)
Shape of y_train: (772796,)
Shape of y_test: (193199,)


**Что делаем?**


Обучим модели KNNRegressor и LinearRegression и оценим их производительность с помощью метрик MSE, MAE и RMSE.


In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Для ускорения работы берём небольшой поднабор данных
subset_size = 10000

X_train_subset = X_train[:subset_size]
y_train_subset = y_train[:subset_size]
X_test_subset = X_test[:subset_size]
y_test_subset = y_test[:subset_size]


# Создаём и обучаем кастомный KNN регрессор
knn_regressor = KNNRegressor(n_neighbors=5)
# Метод fit просто запоминает тренировочные данные
knn_regressor.fit(X_train_subset.values, y_train_subset.values)

# Делаем предсказания с помощью KNN
knn_predictions = knn_regressor.predict(X_test_subset.values)

# Оцениваем качество модели KNN с помощью стандартных метрик
knn_mse = mean_squared_error(y_test_subset, knn_predictions)
knn_mae = mean_absolute_error(y_test_subset, knn_predictions)
knn_rmse = np.sqrt(knn_mse)

print("Custom KNNRegressor Performance:")
print(f"  MSE: {knn_mse}") # среднеквадратичная ошибка
print(f"  MAE: {knn_mae}") # средняя абсолютная ошибка
print(f"  RMSE: {knn_rmse}") # корень из MSE — RMSE

#  cоздаём и обучаем кастомную линейную регрессию (SGD)

linear_regressor_sgd = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=1000)
linear_regressor_sgd.fit(X_train_subset.values, y_train_subset.values)

# Делаем предсказания с помощью линейной регрессии
linear_predictions_sgd = linear_regressor_sgd.predict(X_test_subset.values)

# Оцениваем качество линейной регрессии с помощью тех же метрик
linear_mse_sgd = mean_squared_error(y_test_subset, linear_predictions_sgd)
linear_mae_sgd = mean_absolute_error(y_test_subset, linear_predictions_sgd)
linear_rmse_sgd = np.sqrt(linear_mse_sgd)

print("\nCustom LinearRegression (SGD) Performance:")
print(f"  MSE: {linear_mse_sgd}")
print(f"  MAE: {linear_mae_sgd}")
print(f"  RMSE: {linear_rmse_sgd}")

Custom KNNRegressor Performance:
  MSE: 0.0029024056956415034
  MAE: 0.004806880745303721
  RMSE: 0.05387397976427492

Custom LinearRegression (SGD) Performance:
  MSE: 0.0001452174316615005
  MAE: 0.006906774785455684
  RMSE: 0.012050619555089294


## 3. Сравним модели

Проведем эксперименты с написанными методами машинного обучения. Измерим ошибку MSE, MAE, RMSE. Заиспользуем методы KNNRegressor и LinearRegression из библиотеки sklearn, сравним качество Ваших решений и библиотечных.

In [16]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as SklearnLinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

subset_size = 10000

X_train_subset = X_train[:subset_size]
y_train_subset = y_train[:subset_size]
X_test_subset = X_test[:subset_size]
y_test_subset = y_test[:subset_size]

sklearn_knn_regressor = KNeighborsRegressor(n_neighbors=5)
sklearn_knn_regressor.fit(X_train_subset.values, y_train_subset.values)

sklearn_knn_predictions = sklearn_knn_regressor.predict(X_test_subset.values)

sklearn_knn_mse = mean_squared_error(y_test_subset, sklearn_knn_predictions)
sklearn_knn_mae = mean_absolute_error(y_test_subset, sklearn_knn_predictions)
sklearn_knn_rmse = np.sqrt(sklearn_knn_mse)

print("Scikit-learn KNNRegressor Performance:")
print(f"  MSE: {sklearn_knn_mse}")
print(f"  MAE: {sklearn_knn_mae}")
print(f"  RMSE: {sklearn_knn_rmse}")

sklearn_linear_regressor = SklearnLinearRegression()
sklearn_linear_regressor.fit(X_train_subset.values, y_train_subset.values)

sklearn_linear_predictions = sklearn_linear_regressor.predict(X_test_subset.values)

sklearn_linear_mse = mean_squared_error(y_test_subset, sklearn_linear_predictions)
sklearn_linear_mae = mean_absolute_error(y_test_subset, sklearn_linear_predictions)
sklearn_linear_rmse = np.sqrt(sklearn_linear_mse)

print("\nScikit-learn LinearRegression Performance:")
print(f"  MSE: {sklearn_linear_mse}")
print(f"  MAE: {sklearn_linear_mae}")
print(f"  RMSE: {sklearn_linear_rmse}")

Scikit-learn KNNRegressor Performance:
  MSE: 0.002902404573841272
  MAE: 0.004806679419108768
  RMSE: 0.053873969352937714

Scikit-learn LinearRegression Performance:
  MSE: 0.00015283407592741726
  MAE: 0.007318408730453973
  RMSE: 0.012362607974348181


MSE (Mean Squared Error) — среднеквадратичная ошибка. Чем меньше — тем лучше.
Показывает, насколько сильно предсказанные значения в среднем отклоняются от реальных, в квадрате.

RMSE (Root Mean Squared Error) — квадратный корень из MSE, выражается в тех же единицах, что и данные (тут — стандартизированные).
Его часто используют как «среднее отклонение».

MAE (Mean Absolute Error) — средняя абсолютная ошибка, менее чувствительна к выбросам, чем MSE.

In [18]:
import pandas as pd

performance_data = {
    'Model': ['Custom KNNRegressor', 'Custom LinearRegression (SGD)', 'Scikit-learn KNNRegressor', 'Scikit-learn LinearRegression'],
    'MSE': [knn_mse, linear_mse_sgd, sklearn_knn_mse, sklearn_linear_mse],
    'MAE': [knn_mae, linear_mae_sgd, sklearn_knn_mae, sklearn_linear_mae],
    'RMSE': [knn_rmse, linear_rmse_sgd, sklearn_knn_rmse, sklearn_linear_rmse]
}

performance_df = pd.DataFrame(performance_data)

print("Model Performance Comparison:")
display(performance_df)

Model Performance Comparison:


,Model,MSE,MAE,RMSE
0,Custom KNNRegressor,0.002902,0.004807,0.053874
1,Custom LinearRegression (SGD),0.000145,0.006907,0.012051
2,Scikit-learn KNNRegressor,0.002902,0.004807,0.053874
3,Scikit-learn LinearRegression,0.000153,0.007318,0.012363


В результате вывода метрик KNN получаем почти идентичные значения — значит, реализация KNN корректна

При выводе метрик LR разница значений минимальна, значит, градиентный спуск работает правильно

В стандартизированных данных все признаки приведены к диапазону около [-3, 3],
поэтому ошибки типа RMSE = 0.05 кажутся маленькими.
При обратном преобразовании в рубли видно,
что даже небольшое отклонение приводит к большой ошибке в миллионах рублей.
Можно объяснить такое высокой дисперсией цен и наличием выбросов.

KNN показал чувствительность к выбросам (одна из квартир предсказана в 137 млн),
а линейная регрессия — более стабильное, но всё же неидеальное поведение.
В будущем можно улучшить результаты, применив логарифмирование цен и взвешенные KNN.

In [17]:
print("\nSample Predicted vs. Real Prices (Custom KNNRegressor):")
for i in range(5):
    print(f"  Predicted: {knn_predictions[i]:.4f}, Real: {y_test_subset.iloc[i]:.4f}")

print("\nSample Predicted vs. Real Prices (Custom LinearRegression SGD):")
for i in range(5):
    print(f"  Predicted: {linear_predictions_sgd[i]:.4f}, Real: {y_test_subset.iloc[i]:.4f}")


Sample Predicted vs. Real Prices (Custom KNNRegressor):
  Predicted: -0.0073, Real: -0.0060
  Predicted: -0.0084, Real: -0.0087
  Predicted: -0.0013, Real: -0.0067
  Predicted: 0.0013, Real: 0.0048
  Predicted: 0.2026, Real: 0.2358

Sample Predicted vs. Real Prices (Custom LinearRegression SGD):
  Predicted: -0.0081, Real: -0.0060
  Predicted: -0.0090, Real: -0.0087
  Predicted: 0.0082, Real: -0.0067
  Predicted: 0.0005, Real: 0.0048
  Predicted: 0.0776, Real: 0.2358


Займемся тут обратным преобразованием масштабированных цен обратно в реальные рубли, чтобы можно было сравнивать предсказания моделей с исходными данными.

In [19]:
# Находим индекс столбца 'price' среди масштабированных числовых признаков
price_col_index = numerical_cols_to_scale.index('price')

# Восстанавливаем реальные цены для тестового поднабора
# Используем scaler, который был обучен на тренировочных данных
y_test_subset_original = scaler.inverse_transform(df[numerical_cols_to_scale][:subset_size])[:, price_col_index]

# Подготавливаем массив для обратного преобразования предсказаний KNN
# StandardScaler ожидает массив с тем же числом признаков, что использовался ранее

dummy_array_knn[:, price_col_index] = knn_predictions
knn_predictions_original = scaler.inverse_transform(dummy_array_knn)[:, price_col_index]

dummy_array_linear = np.zeros((len(linear_predictions_sgd), len(numerical_cols_to_scale)))
dummy_array_linear[:, price_col_index] = linear_predictions_sgd
linear_predictions_sgd_original = scaler.inverse_transform(dummy_array_linear)[:, price_col_index]


print("Sample Predicted vs. Real Prices (Custom KNNRegressor - Rubles):")
for i in range(5):
    print(f"  Predicted: {int(knn_predictions_original[i]):,}, Real: {int(y_test_subset_original[i]):,}".replace(',', ' '))

print("\nSample Predicted vs. Real Prices (Custom LinearRegression SGD - Rubles):")
for i in range(5):
    print(f"  Predicted: {int(linear_predictions_sgd_original[i]):,}, Real: {int(y_test_subset_original[i]):,}".replace(',', ' '))

Sample Predicted vs. Real Prices (Custom KNNRegressor - Rubles):
  Predicted: 1 904 000  Real: 2 451 300
  Predicted: 1 216 000  Real: 1 450 000
  Predicted: 5 793 840  Real: 10 700 000
  Predicted: 7 525 739  Real: 3 100 000
  Predicted: 137 981 300  Real: 2 500 000

Sample Predicted vs. Real Prices (Custom LinearRegression SGD - Rubles):
  Predicted: 1 420 779  Real: 2 451 300
  Predicted: 805 384  Real: 1 450 000
  Predicted: 11 984 072  Real: 10 700 000
  Predicted: 6 999 112  Real: 3 100 000
  Predicted: 56 970 686  Real: 2 500 000


В целом, модель ловит порядок цен — миллионы рублей, но сильно ошибается на отдельных примерах.

Особенно заметен большой выброс в последней строке — модель предсказала 137 млн вместо 2,5 млн.
Такое может быть если рядом попались выбросы в обучающей выборке (например, элитные квартиры), они сильно искажают прогноз.